# Full audio

## Feature

### Set up

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COSWARA - HYBRID MODEL FULL AUDIO')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [3]:
n_mfcc = 39

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet'

meta_coswara = os.path.join(root, 'full_audio/coswara_metadata_for_label_matching.csv')
meta_coughvid = os.path.join(root, 'full_audio/coughvid_metadata_for_label_matching.csv')


mfcc_coswara = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coswara_features.npy')
mfcc_coughvid_1 = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features_20k.npy')
mfcc_coughvid_2 = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features_20k_40k.npy')
mfcc_coughvid_3 = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features_40k_60k.npy')
mfcc_coughvid_4 = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features_60k_75k.npy')
mfcc_coughvid_5 = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features_9981.npy')


### Feature preparation

#### Metadata

In [4]:
# process metadata
meta_df = pd.read_csv(meta_coswara)
meta_df.head()

,id,age,covid_status,record_date,is_english_proficiency,gender,country,locality,state,is_returning_user,...,is_fatigue,is_sore_throat,is_ischemic_heart_disease,is_asthma,is_others_preexist_conditions,is_chronic_lung_disease,is_neumonia,cough_path,label,error
0,iV3Db6t1T8b7c5HQY2TwxIhjbzD3,28,healthy,2020-04-23,y,male,India,Anantapur,Andhra Pradesh,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,healthy,0
1,AxuYWBN0jFVLINCBqIW5aZmGCdu1,25,healthy,2020-04-20,y,male,India,BENGALURU URBAN,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/AxuYWBN0jFVLINCBqIW5aZmGCdu1/cough-shallow.wav,healthy,0
2,C5eIsssb9GSkaAgIfsHMHeR6fSh1,28,healthy,2020-04-24,y,female,United States,Pittsburgh,Pennsylvania,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/C5eIsssb9GSkaAgIfsHMHeR6fSh1/cough-shallow.wav,healthy,0
3,YjbEAECMBIaZKyfqOvWy5DDImUb2,26,healthy,2020-04-23,y,male,India,Bangalore,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/YjbEAECMBIaZKyfqOvWy5DDImUb2/cough-shallow.wav,healthy,0
4,aGOvk4ji0cVqIzCs1jHnzlw2UEy2,32,healthy,2020-04-22,y,male,India,Nalanda,Bihar,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/aGOvk4ji0cVqIzCs1jHnzlw2UEy2/cough-shallow.wav,healthy,0


In [5]:
logger.info(meta_df.shape)

2022-04-29 01:27:04,150 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - (4465, 39)


In [6]:
mfcc_feature = np.load(mfcc_coswara)
meta_df = pd.read_csv(meta_coswara)
meta_df = meta_df[meta_df['error'] == 0].reset_index(drop=True)

In [7]:
logger.info(meta_df.shape)

2022-04-29 01:27:05,591 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - (4400, 39)


In [8]:
meta_df.label.value_counts().sum()

4400

In [9]:
mfcc_feature[0].shape, len(mfcc_feature), meta_df.shape

((234, 246), 4400, (4400, 39))

In [10]:
DROP_COLS = [
             'cough_path',
             'error',
             'record_date',
             'id',
             'covid_status',
             'test_date'
]

meta_df.isna().sum()

id                                  0
age                                 0
covid_status                        0
record_date                         0
is_english_proficiency              0
gender                              0
country                             0
locality                          471
state                               0
is_returning_user                1335
is_smoker                        3273
is_cold                          3825
is_hypertension                  4042
is_diabetes                      4024
is_cough                         3638
date_of_ct_scan                  4135
has_ctScan                       3510
ct_score                         4135
is_diarrheoa                     4332
is_fever                         3965
is_loss_of_smell                 4153
is_muscle_pain                   4031
test_type                        3657
test_date                        3657
test_status                      2772
is_using_mask                    2040
vaccination_

In [11]:
# fill na with unknown value
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

id                               0
age                              0
covid_status                     0
record_date                      0
is_english_proficiency           0
gender                           0
country                          0
locality                         0
state                            0
is_returning_user                0
is_smoker                        0
is_cold                          0
is_hypertension                  0
is_diabetes                      0
is_cough                         0
date_of_ct_scan                  0
has_ctScan                       0
ct_score                         0
is_diarrheoa                     0
is_fever                         0
is_loss_of_smell                 0
is_muscle_pain                   0
test_type                        0
test_date                        0
test_status                      0
is_using_mask                    0
vaccination_status               0
is_breathing_difficulty          0
is_others_resp      

In [12]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)
logger.info(meta_df.shape)

2022-04-29 01:27:05,675 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - (4400, 33)


In [13]:
# replace label with binary numbers
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'positive_mild': 1,
    'no_resp_illness_exposed': 1,
    'resp_illness_not_identified': 0,
    'positive_moderate': 1,
    'recovered_full': 0,
    'positive_asymp': 1
})
meta_df.label.value_counts()

0    3194
1    1206
Name: label, dtype: int64

In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label', 'age'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [15]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

In [16]:
mfcc_feature = np.load(mfcc_coswara)

##### 2d

In [17]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [18]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-04-29 01:27:07,506 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - 1206, 3194


In [19]:
# get features for each class
feature_0 = mfcc_feature[index_0]
feature_1 = mfcc_feature[index_1]

In [20]:
print(len(feature_1))

1206


In [21]:
del mfcc_feature

In [22]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:2555], feature_1[:965]), axis=0)
  val = np.concatenate((feature_0[2555:2875], feature_1[965:1086]), axis=0)
  test = np.concatenate((feature_0[2875:], feature_1[1086:]), axis=0)

  y_train = pd.concat([label0[:2555], label1[:965]], ignore_index=True)
  y_val = pd.concat([label0[2555:2875], label1[965:1086]], ignore_index=True)
  y_test = pd.concat([label0[2875:], label1[1086:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [23]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(3520, 234, 246)


In [24]:
y_train_2d

array([0, 0, 0, ..., 1, 1, 1])

In [25]:
del feature_0
del feature_1

##### 1d

In [26]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [27]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [28]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d, random_state=42)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d, random_state=42)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d, random_state=42)

## Model

In [ ]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(3520, 234, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(3520, 32, 1)

In [ ]:
X_train_1d.shape

(3520, 32, 1)

In [ ]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(merged_model, to_file='model_final.png')
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
from datetime import datetime
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

Epoch 1/40
440/440 [==============================] - 49s 73ms/step - loss: 0.6723 - auc: 0.7203 - val_loss: 0.6313 - val_auc: 0.6736
Epoch 2/40
440/440 [==============================] - 31s 71ms/step - loss: 0.5917 - auc: 0.7575 - val_loss: 0.5784 - val_auc: 0.7701
Epoch 3/40
440/440 [==============================] - 31s 71ms/step - loss: 0.5948 - auc: 0.7545 - val_loss: 0.5993 - val_auc: 0.7236
Epoch 4/40
440/440 [==============================] - 32s 73ms/step - loss: 0.5851 - auc: 0.7597 - val_loss: 0.5882 - val_auc: 0.7840
Epoch 5/40
440/440 [==============================] - 31s 71ms/step - loss: 0.5721 - auc: 0.7753 - val_loss: 0.5910 - val_auc: 0.7419
Epoch 6/40
440/440 [==============================] - 31s 71ms/step - loss: 0.5651 - auc: 0.7840 - val_loss: 0.5871 - val_auc: 0.7816
Epoch 7/40
440/440 [==============================] - 31s 71ms/step - loss: 0.5584 - auc: 0.7875 - val_loss: 0.5796 - val_auc: 0.7663
Epoch 8/40
440/440 [==============================] - 31s 71ms

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_39_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_39_hybrid_model_40_8_0.001/assets


Epoch 1/40
220/220 [==============================] - 33s 133ms/step - loss: 0.5205 - auc: 0.8218 - val_loss: 0.7226 - val_auc: 0.6027
Epoch 2/40
220/220 [==============================] - 29s 132ms/step - loss: 0.5099 - auc: 0.8292 - val_loss: 0.6478 - val_auc: 0.7775
Epoch 3/40
220/220 [==============================] - 30s 135ms/step - loss: 0.5042 - auc: 0.8336 - val_loss: 0.6059 - val_auc: 0.8103
Epoch 4/40
220/220 [==============================] - 29s 130ms/step - loss: 0.4936 - auc: 0.8422 - val_loss: 0.6037 - val_auc: 0.7428
Epoch 5/40
220/220 [==============================] - 29s 131ms/step - loss: 0.4826 - auc: 0.8492 - val_loss: 0.5399 - val_auc: 0.8098
Epoch 6/40
220/220 [==============================] - 29s 130ms/step - loss: 0.4673 - auc: 0.8597 - val_loss: 0.6207 - val_auc: 0.7605
Epoch 7/40
220/220 [==============================] - 29s 130ms/step - loss: 0.4455 - auc: 0.8738 - val_loss: 0.6475 - val_auc: 0.7747
Epoch 8/40
14/14 [==============================] - 1s 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_39_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_39_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_39_hybrid_model_40_64_0.001/assets


Epoch 1/40
28/28 [==============================] - 30s 950ms/step - loss: 0.0419 - auc: 0.9970 - val_loss: 2.1338 - val_auc: 0.7474
Epoch 2/40
28/28 [==============================] - 27s 955ms/step - loss: 0.0229 - auc: 0.9997 - val_loss: 1.9770 - val_auc: 0.7308
Epoch 3/40
28/28 [==============================] - 26s 941ms/step - loss: 0.0147 - auc: 0.9999 - val_loss: 2.0294 - val_auc: 0.7404
Epoch 4/40
28/28 [==============================] - 27s 963ms/step - loss: 0.0148 - auc: 0.9999 - val_loss: 2.1151 - val_auc: 0.7575
Epoch 5/40
28/28 [==============================] - 27s 958ms/step - loss: 0.0129 - auc: 0.9999 - val_loss: 2.1696 - val_auc: 0.7516
Epoch 6/40
28/28 [==============================] - 26s 942ms/step - loss: 0.0148 - auc: 0.9996 - val_loss: 2.1630 - val_auc: 0.7536
Epoch 7/40
28/28 [==============================] - 26s 944ms/step - loss: 0.0125 - auc: 0.9999 - val_loss: 2.2318 - val_auc: 0.7506
Epoch 8/40
28/28 [==============================] - 26s 937ms/step - 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_39_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_39_hybrid_model_40_128_0.001/assets


## Standard scaler model

In [ ]:
X_train_2d.shape

(3520, 234, 246)

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(3520, 234, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(3520, 32, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([3520, 2])

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
LEARNING_RATE = 0.001
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5


opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = os.path.join(SAVED_MODEL_PATH, '/coswara_melme_hybrid_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

Epoch 1/40
440/440 [==============================] - 42s 76ms/step - loss: 0.7002 - auc: 0.7158 - val_loss: 0.6507 - val_auc: 0.6724
Epoch 2/40
440/440 [==============================] - 31s 71ms/step - loss: 0.6057 - auc: 0.7374 - val_loss: 0.5723 - val_auc: 0.7711
Epoch 3/40
440/440 [==============================] - 31s 70ms/step - loss: 0.5852 - auc: 0.7628 - val_loss: 1.1090 - val_auc: 0.7373
Epoch 4/40
440/440 [==============================] - 31s 70ms/step - loss: 0.5721 - auc: 0.7765 - val_loss: 1.8287 - val_auc: 0.7239
Epoch 5/40
440/440 [==============================] - 31s 71ms/step - loss: 0.5658 - auc: 0.7843 - val_loss: 0.5709 - val_auc: 0.7784
Epoch 6/40
440/440 [==============================] - 31s 69ms/step - loss: 0.5593 - auc: 0.7906 - val_loss: 0.5826 - val_auc: 0.7704
Epoch 7/40
440/440 [==============================] - 31s 71ms/step - loss: 0.5545 - auc: 0.7947 - val_loss: 0.5626 - val_auc: 0.7907
Epoch 8/40
440/440 [==============================] - 31s 69ms

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mfcc_39_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mfcc_39_hybrid_model_40_8_0.001/assets


Epoch 1/40
220/220 [==============================] - 33s 134ms/step - loss: 0.4374 - auc: 0.8790 - val_loss: 0.5871 - val_auc: 0.7931
Epoch 2/40
220/220 [==============================] - 29s 132ms/step - loss: 0.3914 - auc: 0.9035 - val_loss: 0.7153 - val_auc: 0.7223
Epoch 3/40
220/220 [==============================] - 29s 130ms/step - loss: 0.3382 - auc: 0.9286 - val_loss: 0.6932 - val_auc: 0.7756
Epoch 4/40
220/220 [==============================] - 29s 131ms/step - loss: 0.3206 - auc: 0.9354 - val_loss: 0.8525 - val_auc: 0.7667
Epoch 5/40
220/220 [==============================] - 29s 131ms/step - loss: 0.2807 - auc: 0.9495 - val_loss: 0.9280 - val_auc: 0.6681
Epoch 6/40
14/14 [==============================] - 1s 42ms/step - loss: 0.9421 - auc: 0.6543


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mfcc_39_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mfcc_39_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mfcc_39_hybrid_model_40_64_0.001/assets


Epoch 1/40
28/28 [==============================] - 30s 954ms/step - loss: 0.0127 - auc: 0.9999 - val_loss: 3.0457 - val_auc: 0.6790
Epoch 2/40
28/28 [==============================] - 27s 953ms/step - loss: 0.0130 - auc: 0.9996 - val_loss: 3.0263 - val_auc: 0.6872
Epoch 3/40
28/28 [==============================] - 26s 948ms/step - loss: 0.0128 - auc: 0.9999 - val_loss: 3.0043 - val_auc: 0.6954
Epoch 4/40
28/28 [==============================] - 27s 954ms/step - loss: 0.0129 - auc: 0.9999 - val_loss: 3.0611 - val_auc: 0.6950
Epoch 5/40
28/28 [==============================] - 26s 947ms/step - loss: 0.0116 - auc: 0.9999 - val_loss: 3.0527 - val_auc: 0.6966
Epoch 6/40
28/28 [==============================] - 26s 938ms/step - loss: 0.0114 - auc: 0.9999 - val_loss: 3.0873 - val_auc: 0.7006
Epoch 7/40
28/28 [==============================] - 26s 944ms/step - loss: 0.0127 - auc: 0.9999 - val_loss: 3.0821 - val_auc: 0.6998
Epoch 8/40
28/28 [==============================] - 26s 950ms/step - 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mfcc_39_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mfcc_39_hybrid_model_40_128_0.001/assets


## SMOTE

In [30]:
X_train_2d.shape

(3520, 234, 246)

In [31]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(5110, 234, 246, 1)

In [32]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(5110, 32, 1)

In [33]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [34]:
y_train_2d.shape

TensorShape([5110, 2])

In [35]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [36]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [37]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [38]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [39]:
fp = os.path.join(SAVED_MODEL_PATH, '/coswara_hybrid_mel_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [40]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

Epoch 1/40
639/639 [==============================] - 57s 64ms/step - loss: 0.6998 - auc: 0.6537 - val_loss: 1.6814 - val_auc: 0.2872
Epoch 2/40
639/639 [==============================] - 40s 63ms/step - loss: 0.5742 - auc: 0.7758 - val_loss: 0.7567 - val_auc: 0.7744
Epoch 3/40
639/639 [==============================] - 40s 63ms/step - loss: 0.5440 - auc: 0.8045 - val_loss: 0.8019 - val_auc: 0.7789
Epoch 4/40
639/639 [==============================] - 39s 61ms/step - loss: 0.5280 - auc: 0.8180 - val_loss: 0.6978 - val_auc: 0.7558
Epoch 5/40
639/639 [==============================] - 39s 62ms/step - loss: 0.4988 - auc: 0.8378 - val_loss: 1.3754 - val_auc: 0.7586
Epoch 6/40
639/639 [==============================] - 39s 62ms/step - loss: 0.4807 - auc: 0.8509 - val_loss: 0.6006 - val_auc: 0.7606
Epoch 7/40
639/639 [==============================] - 40s 63ms/step - loss: 0.4637 - auc: 0.8618 - val_loss: 0.5693 - val_auc: 0.7870
Epoch 8/40
639/639 [==============================] - 39s 62ms

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_39_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_39_hybrid_model_40_8_0.001/assets


Epoch 1/40
320/320 [==============================] - 35s 107ms/step - loss: 0.0916 - auc: 0.9916 - val_loss: 1.2664 - val_auc: 0.6770
Epoch 2/40
320/320 [==============================] - 34s 106ms/step - loss: 0.0633 - auc: 0.9945 - val_loss: 1.3624 - val_auc: 0.6896
Epoch 3/40
320/320 [==============================] - 34s 106ms/step - loss: 0.0532 - auc: 0.9954 - val_loss: 1.8445 - val_auc: 0.7116
Epoch 4/40
320/320 [==============================] - 34s 107ms/step - loss: 0.0672 - auc: 0.9950 - val_loss: 1.5880 - val_auc: 0.7293
Epoch 5/40
320/320 [==============================] - 34s 106ms/step - loss: 0.0754 - auc: 0.9929 - val_loss: 1.5193 - val_auc: 0.7523
Epoch 6/40
320/320 [==============================] - 34s 106ms/step - loss: 0.0579 - auc: 0.9953 - val_loss: 1.2563 - val_auc: 0.7381
Epoch 7/40
320/320 [==============================] - 34s 106ms/step - loss: 0.0562 - auc: 0.9961 - val_loss: 1.3860 - val_auc: 0.6830
Epoch 8/40
320/320 [==============================] - 3

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_39_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_39_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_39_hybrid_model_40_64_0.001/assets


Epoch 1/40
40/40 [==============================] - 31s 751ms/step - loss: 0.0136 - auc: 0.9999 - val_loss: 3.6633 - val_auc: 0.6874
Epoch 2/40
40/40 [==============================] - 29s 732ms/step - loss: 0.0135 - auc: 0.9999 - val_loss: 3.6861 - val_auc: 0.6922
Epoch 3/40
40/40 [==============================] - 29s 727ms/step - loss: 0.0126 - auc: 0.9999 - val_loss: 3.6756 - val_auc: 0.6900
Epoch 4/40
40/40 [==============================] - 29s 735ms/step - loss: 0.0139 - auc: 0.9998 - val_loss: 3.6947 - val_auc: 0.6928
Epoch 5/40
40/40 [==============================] - 29s 719ms/step - loss: 0.0127 - auc: 0.9999 - val_loss: 3.7051 - val_auc: 0.6966
Epoch 6/40
40/40 [==============================] - 29s 729ms/step - loss: 0.0133 - auc: 0.9999 - val_loss: 3.7395 - val_auc: 0.6939
Epoch 7/40
40/40 [==============================] - 29s 725ms/step - loss: 0.0133 - auc: 0.9999 - val_loss: 3.7341 - val_auc: 0.6932
Epoch 8/40
40/40 [==============================] - 29s 726ms/step - 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_39_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_39_hybrid_model_40_128_0.001/assets


## Evaluation

In [29]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [30]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [31]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [32]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 3s 35ms/step - loss: 2.9584 - auc: 0.7399


[2.958449363708496, 0.7399296760559082]

In [33]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 1s 29ms/step - loss: 2.6142 - auc: 0.7421


[2.614184856414795, 0.7420830726623535]

In [34]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 1.9304 - auc: 0.6907


[1.9303838014602661, 0.6907109618186951]

In [35]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 0.9421 - auc: 0.6543


[0.9420527219772339, 0.6542696356773376]

In [36]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 0.4693 - auc: 0.8635


[0.4692665636539459, 0.8634737133979797]

### Raw features

In [37]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [38]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 29ms/step - loss: 2.1085 - auc: 0.7456


[2.1084933280944824, 0.7456271052360535]

In [39]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 2.8421 - auc: 0.7616


[2.8420777320861816, 0.7616346478462219]

In [40]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 29ms/step - loss: 1.0956 - auc: 0.7072


[1.0955880880355835, 0.7071725726127625]

In [41]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 0.8718 - auc: 0.7898


[0.8718373775482178, 0.7898309826850891]

In [42]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 0.5596 - auc: 0.7940


[0.5595739483833313, 0.7940132021903992]

### SMOTE

In [43]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 3.4994 - auc: 0.6984


[3.4993813037872314, 0.6984397172927856]

In [44]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 29ms/step - loss: 3.4147 - auc: 0.6925


[3.4147441387176514, 0.6924828886985779]

In [45]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 3.0060 - auc: 0.7003


[3.0060200691223145, 0.7003128528594971]

In [46]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 2.6199 - auc: 0.5671


[2.6199002265930176, 0.5671359300613403]

In [47]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 30ms/step - loss: 0.8067 - auc: 0.7806


[0.8067176342010498, 0.7806103825569153]

# Chunks
* potentials: use `num_chunk` to explore as long audio influence the model'accuracy (the more `num_chunk` the longer the audio)

## Feature

### Set up

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COSWARA - HYBRID MODEL CHUNKS')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [ ]:
n_mfcc = 39

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet'
data_root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing'

meta_coswara = os.path.join(data_root, 'cough_chunking/coswara/coswara_chunk_metadata.csv')
meta_coughvid = os.path.join(data_root, 'cough_chunking/coughvid/coughvid_chunk_metadata.csv')

mel_coswara = os.path.join(root, 'chunks/melspectrogram_coswara_features.npy')
mel_coughvid_1 = os.path.join(root, 'chunks/melspectrogram_coughvid_features_20k.npy')
mel_coughvid_2 = os.path.join(root, 'chunks/melspectrogram_coughvid_features_20k_40k.npy')
mel_coughvid_3 = os.path.join(root, 'chunks/melspectrogram_coughvid_features_40k_60k.npy')
mel_coughvid_4 = os.path.join(root, 'chunks/melspectrogram_coughvid_features_60k_75k.npy')
mel_coughvid_5 = os.path.join(root, 'chunks/melspectrogram_coughvid_features_9981.npy')

mfcc_coswara = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coswara_features.npy')
mfcc_coughvid_1 = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_20k.npy')
mfcc_coughvid_2 = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_20k_40k.npy')
mfcc_coughvid_3 = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_40k_60k.npy')
mfcc_coughvid_4 = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_60k_75k.npy')
mfcc_coughvid_5 = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_9981.npy')


### Feature preparation

#### Metadata

In [ ]:
# process metadata
meta_df = pd.read_csv(meta_coswara)
meta_df.head()

,uuid,age,record_date,is_english_proficiency,gender,country,locality,state,is_returning_user,is_smoker,...,is_sore_throat,is_ischemic_heart_disease,is_asthma,is_others_preexist_conditions,is_chronic_lung_disease,is_neumonia,label,sr_cough,chunk_path,num_chunk
0,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-0,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-0.wav,8
1,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-1,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-1.wav,8
2,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-2,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-2.wav,8
3,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-3,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-3.wav,8
4,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-4,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-4.wav,8


In [ ]:
logger.info(meta_df.shape)

2022-04-28 04:10:14,768 - COSWARA - HYBRID MODEL CHUNKS - INFO - (22878, 39)


In [ ]:
DROP_COLS = [
             'sr_cough',
             'chunk_path',
             'record_date',
             'uuid',
             'test_date'
]

meta_df.isna().sum()

uuid                                 0
age                                  0
record_date                          0
is_english_proficiency               0
gender                               0
country                              0
locality                          2295
state                                0
is_returning_user                 7281
is_smoker                        17592
is_cold                          20117
is_hypertension                  21030
is_diabetes                      20985
is_cough                         19040
date_of_ct_scan                  21754
has_ctScan                       18880
ct_score                         21754
is_diarrheoa                     22514
is_fever                         20704
is_loss_of_smell                 21621
is_muscle_pain                   21229
test_type                        19560
test_date                        19560
test_status                      14906
is_using_mask                    11268
vaccination_status       

In [ ]:
# fill na with unknown value
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

uuid                             0
age                              0
record_date                      0
is_english_proficiency           0
gender                           0
country                          0
locality                         0
state                            0
is_returning_user                0
is_smoker                        0
is_cold                          0
is_hypertension                  0
is_diabetes                      0
is_cough                         0
date_of_ct_scan                  0
has_ctScan                       0
ct_score                         0
is_diarrheoa                     0
is_fever                         0
is_loss_of_smell                 0
is_muscle_pain                   0
test_type                        0
test_date                        0
test_status                      0
is_using_mask                    0
vaccination_status               0
is_breathing_difficulty          0
is_others_resp                   0
is_fatigue          

In [ ]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)
logger.info(meta_df.shape)

2022-04-28 04:10:14,919 - COSWARA - HYBRID MODEL CHUNKS - INFO - (22878, 34)


In [ ]:
# replace label with binary numbers
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'positive_mild': 1,
    'no_resp_illness_exposed': 1,
    'resp_illness_not_identified': 0,
    'positive_moderate': 1,
    'recovered_full': 0,
    'positive_asymp': 1
})
meta_df.label.value_counts()

0    16871
1     6007
Name: label, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label', 'age'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [ ]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

In [ ]:
mfcc_feature = np.load(mfcc_coswara)

##### 2d

In [ ]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [ ]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-04-28 04:10:33,652 - COSWARA - HYBRID MODEL CHUNKS - INFO - 6007, 16871


In [ ]:
# get features for each class
feature_0 = mfcc_feature[index_0]
feature_1 = mfcc_feature[index_1]

In [ ]:
print(len(feature_1))

6007


In [ ]:
del mfcc_feature

In [ ]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:13504], feature_1[:4803]), axis=0)
  val = np.concatenate((feature_0[13504:15186], feature_1[4803:5408]), axis=0)
  test = np.concatenate((feature_0[15186:], feature_1[5408:]), axis=0)

  y_train = pd.concat([label0[:13504], label1[:4803]], ignore_index=True)
  y_val = pd.concat([label0[13504:15186], label1[4803:5408]], ignore_index=True)
  y_test = pd.concat([label0[15186:], label1[5408:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [ ]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(18307, 234, 236)


In [ ]:
y_train_2d

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
del feature_0
del feature_1

##### 1d

In [ ]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [ ]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [ ]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d)

## Model

In [ ]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(18307, 234, 236, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(18307, 33, 1)

In [ ]:
X_train_1d.shape

(18307, 33, 1)

In [ ]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)

Epoch 1/40
144/144 [==============================] - 122s 831ms/step - loss: 0.0719 - auc: 0.9932 - val_loss: 2.0777 - val_auc: 0.7219
Epoch 2/40
144/144 [==============================] - 121s 841ms/step - loss: 0.0122 - auc: 0.9993 - val_loss: 2.5715 - val_auc: 0.7342
Epoch 3/40
144/144 [==============================] - 122s 845ms/step - loss: 0.0347 - auc: 0.9971 - val_loss: 2.0882 - val_auc: 0.7567
Epoch 4/40
144/144 [==============================] - 122s 844ms/step - loss: 0.0332 - auc: 0.9978 - val_loss: 2.1693 - val_auc: 0.7585
Epoch 5/40
144/144 [==============================] - 121s 841ms/step - loss: 0.3060 - auc: 0.9416 - val_loss: 0.9621 - val_auc: 0.7670
Epoch 6/40
144/144 [==============================] - 122s 849ms/step - loss: 0.1250 - auc: 0.9867 - val_loss: 1.3904 - val_auc: 0.6708
Epoch 7/40
144/144 [==============================] - 122s 848ms/step - loss: 0.0362 - auc: 0.9979 - val_loss: 1.8481 - val_auc: 0.7342
Epoch 8/40
144/144 [============================

In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

72/72 [==============================] - 3s 37ms/step - loss: 1.8030 - auc: 0.7436


[1.8030426502227783, 0.7436456680297852]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coswara_mfcc_39_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coswara_mfcc_39_hybrid_model_40_128_0.001/assets


## Standard scaler model

In [ ]:
X_train_2d.shape

(18307, 234, 236)

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(18307, 234, 236, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(18307, 33, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([18307, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=256, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 256, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = os.path.join(SAVED_MODEL_PATH, f'/coswara_mfcc_{n_mfcc}_hybrid_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)

Epoch 1/40
144/144 [==============================] - 126s 857ms/step - loss: 0.0063 - auc: 0.9994 - val_loss: 3.0958 - val_auc: 0.7216
Epoch 2/40
144/144 [==============================] - 123s 856ms/step - loss: 0.0012 - auc: 0.9999 - val_loss: 4.7791 - val_auc: 0.7089
Epoch 3/40
144/144 [==============================] - 122s 851ms/step - loss: 7.4000e-04 - auc: 0.9999 - val_loss: 5.3399 - val_auc: 0.7035
Epoch 4/40
144/144 [==============================] - 123s 855ms/step - loss: 5.3639e-04 - auc: 0.9999 - val_loss: 5.3831 - val_auc: 0.7036
Epoch 5/40
144/144 [==============================] - 123s 854ms/step - loss: 7.7669e-04 - auc: 0.9999 - val_loss: 5.4218 - val_auc: 0.7064
Epoch 6/40
144/144 [==============================] - 123s 855ms/step - loss: 4.9386e-04 - auc: 0.9999 - val_loss: 6.5593 - val_auc: 0.6972


In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

72/72 [==============================] - 3s 37ms/step - loss: 6.5006 - auc: 0.7277


[6.500614643096924, 0.7276625037193298]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'std_coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coswara_mfcc_39_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coswara_mfcc_39_hybrid_model_40_128_0.001/assets


## SMOTE

In [ ]:
X_train_2d.shape

(18307, 234, 236)

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(27008, 234, 236, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(27008, 33, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([27008, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=256, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 256, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = os.path.join(SAVED_MODEL_PATH, f'/coswara_hybrid_mfcc_{n_mfcc}_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)

Epoch 1/40
211/211 [==============================] - 144s 661ms/step - loss: 0.0944 - auc: 0.9909 - val_loss: 2.2170 - val_auc: 0.7553
Epoch 2/40
211/211 [==============================] - 141s 668ms/step - loss: 0.0587 - auc: 0.9949 - val_loss: 2.4939 - val_auc: 0.7528
Epoch 3/40
211/211 [==============================] - 140s 666ms/step - loss: 0.0495 - auc: 0.9959 - val_loss: 2.0866 - val_auc: 0.5955
Epoch 4/40
211/211 [==============================] - 140s 664ms/step - loss: 0.0723 - auc: 0.9933 - val_loss: 1.9000 - val_auc: 0.7473
Epoch 5/40
211/211 [==============================] - 141s 668ms/step - loss: 0.0581 - auc: 0.9951 - val_loss: 1.9106 - val_auc: 0.6911
Epoch 6/40
211/211 [==============================] - 141s 667ms/step - loss: 0.0492 - auc: 0.9959 - val_loss: 2.4250 - val_auc: 0.6446


In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d], verbose=1)

72/72 [==============================] - 2s 28ms/step - loss: 2.0598 - auc: 0.6951


[2.0598015785217285, 0.6951145529747009]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coswara_mfcc_39_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coswara_mfcc_39_hybrid_model_40_128_0.001/assets


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs/fit

## Evaluation

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 15s 34ms/step - loss: 6.5006 - auc: 0.7277


[6.500614643096924, 0.7276625037193298]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 3s 28ms/step - loss: 2.4730 - auc: 0.7349


[2.4729669094085693, 0.7348905801773071]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 3s 28ms/step - loss: 1.5812 - auc: 0.7626


[1.581168532371521, 0.7625900506973267]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 3s 29ms/step - loss: 1.0228 - auc: 0.7659


[1.022756576538086, 0.7658887505531311]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 3s 28ms/step - loss: 0.5883 - auc: 0.7963


[0.5883069634437561, 0.7962830066680908]

### Raw features

In [ ]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 28ms/step - loss: 1.8030 - auc: 0.7436


[1.8030424118041992, 0.7436456680297852]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 28ms/step - loss: 1.3736 - auc: 0.7367


[1.373632788658142, 0.736656665802002]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 28ms/step - loss: 0.8905 - auc: 0.7531


[0.890545129776001, 0.753103494644165]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 28ms/step - loss: 0.5805 - auc: 0.8032


[0.5805359482765198, 0.8032351732254028]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 28ms/step - loss: 0.5417 - auc: 0.8006


[0.5417167544364929, 0.8005975484848022]

### SMOTE

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 28ms/step - loss: 2.0598 - auc: 0.6951


[2.059802293777466, 0.6951145529747009]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 29ms/step - loss: 1.0394 - auc: 0.7281


[1.0394107103347778, 0.7281423211097717]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 29ms/step - loss: 1.5698 - auc: 0.7416


[1.5698140859603882, 0.7416442036628723]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 28ms/step - loss: 1.1873 - auc: 0.7639


[1.1873074769973755, 0.7639330625534058]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 28ms/step - loss: 0.6565 - auc: 0.7582


[0.6565366983413696, 0.7581567168235779]